In [ ]:
# ==============================================================================
# FAZA 1: PRIPREMA OKRUŽENJA
# ==============================================================================
# Instaliramo ključne biblioteke.
# 'transformers' i 'datasets' su iz Hugging Face ekosistema.
# 'sentencepiece' je potreban za mT5 tokenizer.
# 'evaluate' i 'rouge_score' su za evaluaciju.
!pip install transformers[torch] datasets sentencepiece evaluate rouge_score -q

import pandas as pd
import os
import glob # Za lako pronalaženje svih CSV datoteka
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import torch
import evaluate # Biblioteka za evaluaciju od Hugging Face

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

# Provjerimo da li je GPU dostupan, što je ključno za brzinu treniranja
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Koristi se uređaj: {device}")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.8 MB/s eta 0:00:00
Koristi se uređaj: cuda


In [3]:
# ==============================================================================
# FAZA 2: UČITAVANJE I SPAJANJE VIŠE JSON FAJLOVA
# ==============================================================================
import pandas as pd
from google.colab import files
import glob
import os

# Kreiramo privremeni folder za upload
upload_dir = 'json_uploads'
if not os.path.exists(upload_dir):
    os.makedirs(upload_dir)

# Prelazimo u taj direktorij da bi uploadovani fajlovi završili tu
os.chdir(upload_dir)

# Pitamo korisnika da uploaduje sve željene JSON fajlove
print(f"Molimo, uploadujte SVE JSON fajlove koje želite koristiti za trening.")
print("Možete ih odabrati sve odjednom.")
uploaded_files = files.upload()

# Vraćamo se u glavni direktorij
os.chdir('..')

# Provjeravamo da li je išta uploadovano
if not uploaded_files:
    print("\nNijedan fajl nije uploadovan. Prekidam izvršavanje.")
else:
    print(f"\nUspješno ste uploadovali {len(uploaded_files)} fajlova.")

    # Lista za spremanje svih DataFrame-ova
    df_list = []

    # Pronalazimo sve JSON fajlove u upload folderu
    json_path = os.path.join(upload_dir, "*.json")
    all_json_files = glob.glob(json_path)

    # Učitavamo svaki JSON fajl i dodajemo ga u listu
    for file_path in all_json_files:
        try:
            print(f"Učitavam fajl: {os.path.basename(file_path)}...")
            temp_df = pd.read_json(file_path, encoding='utf-8')
            df_list.append(temp_df)
            print(f" -> Učitano {len(temp_df)} redova.")
        except Exception as e:
            print(f"GREŠKA pri učitavanju fajla {file_path}: {e}")

    # Spajamo sve DataFrame-ove u jedan veliki
    if df_list:
        df = pd.concat(df_list, ignore_index=True)
        print("\nSvi podaci su uspješno spojeni.")
        print(f"UKUPAN BROJ ČLANAKA ZA TRENING: {len(df)}")
        print("\nKolone u finalnom DataFrame-u:")
        print(df.columns)

        # Provjera duplikata na osnovu ID-a
        initial_count = len(df)

        # Koristimo kolonu 'id' (ili 'ID' ako se ponekad pojavljuje velikim slovima)
        id_column = 'id' if 'id' in df.columns else 'ID'
        df.drop_duplicates(subset=[id_column], inplace=True, keep='first')

        final_count = len(df)
        print(f"\nUklonjeno {initial_count - final_count} duplikata na osnovu kolone '{id_column}'.")
        print(f"KONAČAN BROJ JEDINSTVENIH ČLANAKA: {len(df)}")


    else:
        print("\nNijedan DataFrame nije kreiran. Provjerite da li su fajlovi ispravnog formata.")

Molimo, uploadujte SVE JSON fajlove koje želite koristiti za trening.
Možete ih odabrati sve odjednom.


Saving buka_Karikature i stripovi_RA.json to buka_Karikature i stripovi_RA.json
Saving buka_Podcast_RA.json to buka_Podcast_RA.json
Saving buka_TV_RA.json to buka_TV_RA.json
Saving FINALNI_SAZECI.json to FINALNI_SAZECI.json

Uspješno ste uploadovali 4 fajlova.
Učitavam fajl: buka_Karikature i stripovi_RA.json...
 -> Učitano 23 redova.
Učitavam fajl: buka_Podcast_RA.json...
 -> Učitano 139 redova.
Učitavam fajl: FINALNI_SAZECI.json...
 -> Učitano 78893 redova.
Učitavam fajl: buka_TV_RA.json...
 -> Učitano 20 redova.

Svi podaci su uspješno spojeni.
UKUPAN BROJ ČLANAKA ZA TRENING: 79075

Kolone u finalnom DataFrame-u:
Index(['portal', 'kategorija', 'id', 'url', 'datum', 'naslov', 'tekst',
       'ekstraktivna_sumarizacija', 'apstraktivna_sumarizacija',
       'chatgpt_sumarizacija', 'gemini_sumarizacija', 'claude_sumarizacija'],
      dtype='object')

Uklonjeno 0 duplikata na osnovu kolone 'id'.
KONAČAN BROJ JEDINSTVENIH ČLANAKA: 79075


In [4]:
# ==============================================================================
# FAZA 3: PRIPREMA PODATAKA ZA MODEL
# ==============================================================================
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# 3.1. Čišćenje i filtriranje
# Ovdje koristimo tačna imena kolona iz JSON fajla.

# Ovdje biramo koji sažetak želimo koristiti kao cilj (labelu).
# Ja sam ostavio 'claude_sumarizacija' kao preporuku.
summary_columns = ['claude_sumarizacija', 'chatgpt_sumarizacija', 'gemini_sumarizacija', 'ekstraktivna_sumarizacija', 'apstraktivna_sumarizacija']


# Ostavljamo samo potrebne kolone i preimenujemo ih u 'text' i 'summary'
# kako bi ostatak koda radio bez izmjena.
try:
    #df_prepared = df[['tekst', target_summary_column]].copy()
    #df_prepared.rename(columns={'tekst': 'text', target_summary_column: 'summary'}, inplace=True)

    # Uklanjamo redove gdje je tekst ili sažetak prazan/nedostaje (NaN)
    #df_prepared.dropna(subset=['text', 'summary'], inplace=True)
    # Uklanjamo redove gdje tekst ili sažetak imaju manje od 20 karaktera
    #df_prepared = df_prepared[df_prepared['text'].str.len() > 20]
    #df_prepared = df_prepared[df_prepared['summary'].str.len() > 20]

    #print(f"Broj članaka nakon čišćenja: {len(df_prepared)}")

    df_versions = []
    for col in summary_columns:
        if col in df.columns:
            temp_df = df[['tekst', col]].copy()
            temp_df.rename(columns={'tekst': 'text', col: 'summary'}, inplace=True)
            temp_df.dropna(subset=['text', 'summary'], inplace=True)
            df_versions.append(temp_df)

    # Spajamo svih 5 u jedan veliki DataFrame
    df_prepared = pd.concat(df_versions, ignore_index=True)

    # Filtar za minimum dužine
    df_prepared = df_prepared[df_prepared['text'].str.len() > 20]
    df_prepared = df_prepared[df_prepared['summary'].str.len() > 20]

    print(f"Ukupno uzoraka za treniranje sa svim sumarizacijama: {len(df_prepared)}")

    # 3.2. Podjela na train i test skupove (90% train, 10% test)
    train_df, test_df = train_test_split(df_prepared, test_size=0.1, random_state=42)

    print(f"Veličina trening skupa: {len(train_df)}")
    print(f"Veličina test skupa: {len(test_df)}")

    # 3.3. Kreiranje Hugging Face Dataset objekta
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    raw_datasets = DatasetDict({
        'train': train_dataset,
        'test': test_dataset
    })

    print("\nDataset je spreman za tokenizaciju:")
    print(raw_datasets)

except KeyError:
    print("\nGREŠKA: Jedna od kolona ('tekst' ili '" + target_summary_column + "') nije pronađena u DataFrame-u.")
    print("Provjerite da li su imena kolona u kodu ista kao u ispisu iznad.")

Ukupno uzoraka za treniranje sa svim sumarizacijama: 1899
Veličina trening skupa: 1709
Veličina test skupa: 190

Dataset je spreman za tokenizaciju:
DatasetDict({
    train: Dataset({
        features: ['text', 'summary', '__index_level_0__'],
        num_rows: 1709
    })
    test: Dataset({
        features: ['text', 'summary', '__index_level_0__'],
        num_rows: 190
    })
})


In [5]:

# ==============================================================================
# FAZA 4: UČITAVANJE MODELA I TOKENIZERA (ISPRAVLJENA VERZIJA)
# ==============================================================================
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Provjerimo da li je GPU dostupan, što je ključno za brzinu treniranja
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Koristi se uređaj: {device}")


# ISPRAVKA JE OVDJE:
# Koristimo 'google/mt5-small' umjesto pogrešnog imena.
# Ovo je zvanični, javno dostupan model od Google-a.
model_checkpoint = "google/mt5-small"

try:
    # Tokenizer pretvara tekst u brojeve (tokene)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    # Model koji ćemo fino podesiti (fine-tune)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

    print(f"Model '{model_checkpoint}' i njegov tokenizer su uspješno učitani.")

except Exception as e:
    print(f"Došlo je do greške prilikom učitavanja modela: {e}")
    print("\nProvjerite da li je ime modela ('model_checkpoint') tačno i da li imate pristup internetu.")


# ==============================================================================
# FAZA 5: TOKENIZACIJA (ISPRAVLJENA VERZIJA BEZ PREFIKSA)
# ==============================================================================
# Definišemo maksimalne dužine za ulaz (članak) i izlaz (sažetak)
max_input_length = 1024
max_target_length = 128
# UKLONILI SMO 'prefix' VARIJABLU

def preprocess_function(examples):
    # VIŠE NE DODAJEMO PREFIKS. Koristimo direktno tekst.
    #inputs = examples["text"]
    inputs = ["summarize: " + text for text in examples["text"]]

    # Tokeniziramo članke (ulaz)
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Tokeniziramo sažetke (cilj/labela)
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    # Postavljamo tokenizirane sažetke kao 'labels'
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Mapiramo funkciju na cijeli dataset. 'batched=True' ubrzava proces.
# Uklanjamo stare kolone koje nam više ne trebaju.
tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)
print("Dataset je tokeniziran (bez prefiksa).")
print(tokenized_datasets)

Koristi se uređaj: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model 'google/mt5-small' i njegov tokenizer su uspješno učitani.


Map:   0%|          | 0/1709 [00:00<?, ? examples/s]

Map:   0%|          | 0/190 [00:00<?, ? examples/s]

Dataset je tokeniziran (bez prefiksa).
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1709
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 190
    })
})


In [7]:
# ==============================================================================
# FAZA 4: UČITAVANJE MODELA I TOKENIZERA (ISPRAVLJENA VERZIJA)
# ==============================================================================
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Provjerimo da li je GPU dostupan, što je ključno za brzinu treniranja
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Koristi se uređaj: {device}")


# ISPRAVKA JE OVDJE:
# Koristimo 'google/mt5-small' umjesto pogrešnog imena.
# Ovo je zvanični, javno dostupan model od Google-a.
model_checkpoint = "google/mt5-small"

try:
    # Tokenizer pretvara tekst u brojeve (tokene)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    # Model koji ćemo fino podesiti (fine-tune)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

    print(f"Model '{model_checkpoint}' i njegov tokenizer su uspješno učitani.")

except Exception as e:
    print(f"Došlo je do greške prilikom učitavanja modela: {e}")
    print("\nProvjerite da li je ime modela ('model_checkpoint') tačno i da li imate pristup internetu.")


# ==============================================================================
# FAZA 5: TOKENIZACIJA (ISPRAVLJENA VERZIJA BEZ PREFIKSA)
# ==============================================================================
# Definišemo maksimalne dužine za ulaz (članak) i izlaz (sažetak)
max_input_length = 1024
max_target_length = 128
# UKLONILI SMO 'prefix' VARIJABLU

def preprocess_function(examples):
    # VIŠE NE DODAJEMO PREFIKS. Koristimo direktno tekst.
    #inputs = examples["text"]
    inputs = ["summarize: " + text for text in examples["text"]]

    # Tokeniziramo članke (ulaz)
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Tokeniziramo sažetke (cilj/labela)
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    # Postavljamo tokenizirane sažetke kao 'labels'
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Mapiramo funkciju na cijeli dataset. 'batched=True' ubrzava proces.
# Uklanjamo stare kolone koje nam više ne trebaju.
tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)
print("Dataset je tokeniziran (bez prefiksa).")
print(tokenized_datasets)


# ==============================================================================
# FAZA 6: DEFINISANJE ARGUMENATA ZA TRENIRANJE (VERZIJA ZA DUŽI TRENING)
# ==============================================================================
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
import evaluate
import numpy as np

# Postavke za treniranje
batch_size = 4
model_name = "mt5-small-balkan-news-summarizer"
output_dir = f"./{model_name}"

# Objašnjenje strategije:
# Imamo ~180k primjera u trening setu. Jedna epoha (prolazak kroz sve podatke)
# sa batch size 4 bi trajala ~45,000 koraka, što je predugo za Colab.
# Zato ograničavamo trening na fiksni broj koraka ('max_steps').
#
# Biramo 2000 koraka kao dobar kompromis. To znači da će model vidjeti
# 2000 * 4 = 8000 članaka. To je dovoljno da se vide prvi smisleni rezultati,
# a trening bi trebao završiti unutar nekoliko sati.

# Argumenti za treniranje
args = Seq2SeqTrainingArguments(
    output_dir=output_dir,

    # Strategija evaluacije i spremanja:
    # Izvrši evaluaciju i sačuvaj model na svakih 500 koraka.
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,

    # Hiperparametri učenja:
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,

    # Ograničavanje resursa:
    save_total_limit=3,          # Čuva samo 3 zadnja checkpointa (npr. chkpt-1000, 1500, 2000)
    max_steps=2000,              # KLJUČNA PROMJENA: Ograničavamo trening na 2000 koraka

    # Tehnički detalji:
    predict_with_generate=True,  # Neophodno za generisanje teksta tokom evaluacije
    fp16=False,                   # Ubrzava trening na modernim GPU-ovima
    push_to_hub=False,           # Ne objavljujemo model na Hugging Face Hub
    report_to="none",            # Isključujemo logiranje na 'wandb'
)

# Data Collator spaja podatke iz dataseta u 'batch'-eve
# i dinamički ih 'pad'-uje (dopunjava) na istu dužinu.
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Učitavamo ROUGE metriku
rouge = evaluate.load("rouge")

# Funkcija za izračunavanje metrika tokom evaluacije
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Dekodiramo predikcije (generisani tekst)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Zamijenimo -100 (koji se koristi za ignorisanje padding tokena u labelama) sa pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Dekodiramo labele (referentni tekst)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Izračunamo ROUGE skorove
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Dodajemo i prosječnu dužinu generisanih sažetaka kao zanimljivu metriku
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    # Zaokružujemo rezultate na 4 decimale radi preglednosti
    return {k: round(v, 4) for k, v in result.items()}

print("Faza 6 je konfigurisana za duži trening. Spremni za pokretanje Faze 7.")

# ==============================================================================
# FAZA 7: TRENIRANJE
# ==============================================================================
# Kreiramo Trainer objekat
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"], # Koristimo test set za evaluaciju tokom treniranja
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("Počinje treniranje modela...")
trainer.train()


# ==============================================================================
# FAZA 8: EVALUACIJA
# ==============================================================================
print("\nPočinje finalna evaluacija na testnom skupu...")
evaluation_results = trainer.evaluate()
print("\nRezultati evaluacije:")
print(evaluation_results)

Koristi se uređaj: cuda


/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Model 'google/mt5-small' i njegov tokenizer su uspješno učitani.


Map:   0%|          | 0/1709 [00:00<?, ? examples/s]

Map:   0%|          | 0/190 [00:00<?, ? examples/s]

Dataset je tokeniziran (bez prefiksa).
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1709
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 190
    })
})
Faza 6 je konfigurisana za duži trening. Spremni za pokretanje Faze 7.
Počinje treniranje modela...


/tmp/ipython-input-7-2176435267.py:148: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,11.367900,3.240471,0.111900,0.020800,0.094300,0.094100,20.000000
1000,4.906500,3.113930,0.126800,0.042800,0.104900,0.104900,19.089500
1500,4.382700,3.055346,0.127700,0.051800,0.107400,0.107000,18.984200
2000,4.214300,3.053518,0.127800,0.053300,0.106900,0.106200,19.052600



Počinje finalna evaluacija na testnom skupu...



Rezultati evaluacije:
{'eval_loss': 3.0535175800323486, 'eval_rouge1': 0.1278, 'eval_rouge2': 0.0533, 'eval_rougeL': 0.1069, 'eval_rougeLsum': 0.1062, 'eval_gen_len': 19.0526, 'eval_runtime': 30.7562, 'eval_samples_per_second': 6.178, 'eval_steps_per_second': 1.561, 'epoch': 4.672897196261682}


In [8]:
# ==============================================================================
# FAZA 9: UČITAVANJE SAČUVANOG MODELA I TESTIRANJE
# ==============================================================================
from transformers import pipeline
import os

# Putanja do direktorija gdje je Trainer spremao modele
output_dir = "./mt5-small-balkan-news-summarizer"

# Pronalazimo zadnji sačuvani checkpoint, jer on sadrži najbolju verziju modela
try:
    # Listamo sve foldere unutar output direktorija koji počinju sa 'checkpoint-'
    checkpoints = [d for d in os.listdir(output_dir) if d.startswith("checkpoint-")]
    # Sortiramo ih numerički da nađemo zadnji (najveći broj)
    last_checkpoint = sorted(checkpoints, key=lambda x: int(x.split('-')[1]))[-1]
    # Sastavljamo punu putanju do najboljeg modela
    model_path = os.path.join(output_dir, last_checkpoint)
    print(f"Pronađen i koristi se zadnji sačuvani model iz: {model_path}")

except (IndexError, FileNotFoundError):
    print(f"GREŠKA: Nije pronađen nijedan 'checkpoint' u direktoriju '{output_dir}'.")
    print("Provjerite da li je Faza 7 (treniranje) uspješno završena i kreirala foldere.")
    # Postavljamo model_path na None da ne bi došlo do daljih grešaka
    model_path = None


# Ako smo uspješno pronašli putanju, nastavljamo sa testiranjem
if model_path:
    # Učitamo sačuvani model direktno sa putanje
    summarizer = pipeline("summarization", model=model_path, device=0) # device=0 znači da koristi GPU

    # Uzmemo neki članak iz testnog seta za primjer
    # Moramo osigurati da 'test_df' postoji. Ako je sesija prekinuta, moramo ga ponovo kreirati.
    # Ako 'test_df' ne postoji, ova linija će izazvati NameError.
    for test_article_id in range(1, 10):
      try:
          test_article = test_df.iloc[test_article_id]['text']
          reference_summary = test_df.iloc[test_article_id]['summary']

          # Generišemo sažetak
          generated_summary = summarizer(test_article, max_length=150, min_length=30, do_sample=False)[0]['summary_text']

          print("\n--- TEST NA JEDNOM PRIMJERU ---")
          print(f"\nORIGINALNI ČLANAK:\n{test_article[:500]}...")
          print("-" * 30)
          print(f"REFERENTNI SAŽETAK (Claude):\n{reference_summary}")
          print("-" * 30)
          print(f"SAŽETAK NAŠEG MODELA:\n{generated_summary}\n\n\n")

      except NameError:
          print("\nGREŠKA: Varijabla 'test_df' nije definirana. Sesija se vjerojatno prekinula.")
          print("Molimo, ponovo pokrenite ćelije od Faze 2 i Faze 3 da biste ponovo kreirali 'test_df', a zatim se vratite i pokrenite ovu ćeliju ponovo.")
          print("Ne morate ponovo trenirati model (Faza 7).")

Pronađen i koristi se zadnji sačuvani model iz: ./mt5-small-balkan-news-summarizer/checkpoint-2000


Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- TEST NA JEDNOM PRIMJERU ---

ORIGINALNI ČLANAK:
Snežana Kutlešić-Stević doktor je medicine, specijalista pulmologije i baromedicine. Načelnica je Odjeljenja pulmološke rehabilitacije u Zavodu za fizikalnu medicinu i rehabilitaciju „Dr Miroslav Zotović“. Odbornica je u Skupštini grada Banjaluka, članica je Glavnog i Gradskog odbora SNSD-a Banjaluka. Sa Kutlešić-Stević za potal BUKA razgovaramo o kandidaturi za Narodnu skupštinu RS, političkom radu, ženama u politici i drugim temama. Imajući u vidu da ste odbornica u Skupštini grada Banjaluka, ...
------------------------------
REFERENTNI SAŽETAK (Claude):
Snežana Kutlešić-Stević, pulmolog i odbornica u Skupštini grada Banjaluka (SNSD), kandiduje se za Narodnu skupštinu RS na prijedlog mjesnih odbora, nakon što 2018. nije prošla iako je osvojila 6 hiljada glasova; Kampanja za Narodnu skupštinu RS obuhvata šire područje (izborna jedinica 3: Banjaluka, Čelinac, Kotor Varoš, Kneževo, Mrkonjić Grad, Ribnik, Šipovo), što zahtijeva više a

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- TEST NA JEDNOM PRIMJERU ---

ORIGINALNI ČLANAK:
Registar o broju oboljelih od rijetkih bolesti u BiH ne postoji, a i pristup oboljelima razlikuje se na entitetskom nivou. Dok je broj oboljelih od rijetkih bolesti u Federaciji BiH nepoznat, u Republici Srpskoj je oko 500 oboljelih. Osobe koje boluju od rijetkih bolesti u BiH suočavaju se sa brojnim problemima, od uspostavljanja dijagnoze do načina liječenja i dolaska do lijekova. Strategija o rijetkim bolestima u Federaciji BiH je usvojena 2014. godine za period 2014 – 2020. s ciljem, kako se ...
------------------------------
REFERENTNI SAŽETAK (Claude):
* U Bosni i Hercegovini ne postoji centralni registar oboljelih od rijetkih bolesti, a pristup liječenju i dostupnost lijekova se značajno razlikuje između entiteta.  Federacija BiH ima usvojenu strategiju, ali njena implementacija je minimalna, dok Republika Srpska ima nešto bolje razvijen sistem podrške, iako i on nije idealan.

* Osobe oboljele od rijetkih bolesti u BiH suočava

...


--- TEST NA JEDNOM PRIMJERU ---

ORIGINALNI ČLANAK:
Ivanić navodi da je bezbjednosna situacija u biH itekako ugrožena i da je za to najzahvalnija retorika ljudi na vlasti i i politička kriza koja predugo traje. „Ova kriza traje praktićčno od izbora 2018. godine. Imate situaciju da nemate novu vladu Federacije, da institucije BiH ne rade ili rade povremeno, a glavni razlog toga je da tri ključna politička faktora SNSD, SDA i HDZ nisu istinski prihvatila Dejtonski mirovni sporazum. Svako od njih želi da ostvari svoje ciljeve. SDA želi BiH bez entit...
------------------------------
REFERENTNI SAŽETAK (Claude):
Mladen Ivanić ocjenjuje bezbjednosnu situaciju u BiH ugroženom zbog političke krize i retorike vlasti, tvrdeći da ključni politički faktori (SNSD, SDA i HDZ) nisu prihvatili Dejtonski sporazum i nastavljaju "rat u mirnodobskim okolnostima", pri čemu je položaj RS najteži zbog gubitka saveznika. Mladen Ivanić upozorava na moguće incidente zbog zapaljive retorike političkih opcija, 

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- TEST NA JEDNOM PRIMJERU ---

ORIGINALNI ČLANAK:
Iphone aplikacija omogućava čitanje sadržaja portala BUKA, gledanje emisija BUKA te dijeljenje (share) sadržaja putem društvenih mreža. Aplikacija sadrži aktuelne dnevne informacije iz oblasti politike, ekonomije, društva, sporta, kulture i zabave. Ovim smo odlučili da Vam na još jedan način približimo naše sadržaje. BUKA portal bilježi konstantan rast čitanosti, te prema google analytics statistici naše sadržaje dnevno otvori između 10 i 15 hiljada čitalaca. Uskoro pokrećemo i BUKA Andriod apli...
------------------------------
REFERENTNI SAŽETAK (Claude):
Aplikacija BUKA za iPhone omogućuje korisnicima brendiran i modern način pristupa najnovijim vijestima sa portala BUKA. Korisnici mogu čitati članke, gledati emisije i dijeliti sadržaje putem društvenih mreža. Aplikacija nudi raznovrsni sadržaj, uključujući politiku, ekonomiju, društvo, sport i zabavu, te je dostupna za besplatno preuzimanje sada.
------------------------------
SA

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- TEST NA JEDNOM PRIMJERU ---

ORIGINALNI ČLANAK:
U nedjelju, 24. aprila, se očekuje umjereno do pretežno oblačno vrijeme sa kišom ili lokalnim pljuskovima. Prijepodne padavine uglavnom na zapadu i sjeverozapadu Bosne. Poslijepodne u većem dijelu zemlje. U večernjm satima uglavnom u Hercegovini i na jugozapadu Bosne. Jutarnja temperatura od osam do 14, a dnevna od 13 do 19, u Hercegovini i na sjeveroistoku Bosne do 23 stepena. Jutro i prijepodne u ponedjeljak, 25. aprila, pretežno oblačno sa kišom ili lokalnim pljuskovima. Poslijepodne postepen...
------------------------------
REFERENTNI SAŽETAK (Claude):
U nedjelju, 24. aprila, se očekuje umjereno do pretežno oblačno vrijeme sa kišom ili lokalnim pljuskovima.
------------------------------
SAŽETAK NAŠEG MODELA:
<extra_id_0> i prijepodne u Bosni porast naoblake što može usloviti kišu, pljuskove i grmljavinu. Poslijepodne postepeni prestanak padavina i smanjenje oblačnosti. Poslijepodne postepeni prestanak padavina i smanjenje oblač

In [9]:
!zip -r model.zip ./mt5-small-balkan-news-summarizer/checkpoint-2000

from google.colab import files
files.download('model.zip')

  adding: mt5-small-balkan-news-summarizer/checkpoint-2000/ (stored 0%)
  adding: mt5-small-balkan-news-summarizer/checkpoint-2000/scheduler.pt (deflated 56%)
  adding: mt5-small-balkan-news-summarizer/checkpoint-2000/config.json (deflated 48%)
  adding: mt5-small-balkan-news-summarizer/checkpoint-2000/optimizer.pt (deflated 44%)
  adding: mt5-small-balkan-news-summarizer/checkpoint-2000/spiece.model (deflated 46%)
  adding: mt5-small-balkan-news-summarizer/checkpoint-2000/tokenizer.json (deflated 76%)
  adding: mt5-small-balkan-news-summarizer/checkpoint-2000/training_args.bin (deflated 52%)
  adding: mt5-small-balkan-news-summarizer/checkpoint-2000/trainer_state.json (deflated 72%)
  adding: mt5-small-balkan-news-summarizer/checkpoint-2000/tokenizer_config.json (deflated 95%)
  adding: mt5-small-balkan-news-summarizer/checkpoint-2000/model.safetensors (deflated 25%)
  adding: mt5-small-balkan-news-summarizer/checkpoint-2000/special_tokens_map.json (deflated 73%)
  adding: mt5-small-b

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>